In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [3]:
path = "C:/Users/hjhhi/Desktop/Data/"
train = pd.read_csv(os.path.join(path, "train.csv"))
test = pd.read_csv(os.path.join(path, "test.csv"))
sample_submission = pd.read_csv(os.path.join(path, "sample_submission.csv"))

In [4]:
train.head(5)

,ID,COMPONENT_ARBITRARY,ANONYMOUS_1,YEAR,SAMPLE_TRANSFER_DAY,ANONYMOUS_2,AG,AL,B,BA,...,U25,U20,U14,U6,U4,V,V100,V40,ZN,Y_LABEL
0,TRAIN_00000,COMPONENT3,1486,2011,7,200,0,3,93,0,...,NaN,NaN,NaN,NaN,NaN,0,NaN,154.0,75,0
1,TRAIN_00001,COMPONENT2,1350,2021,51,375,0,2,19,0,...,2.0,4.0,6.0,216.0,1454.0,0,NaN,44.0,652,0
2,TRAIN_00002,COMPONENT2,2415,2015,2,200,0,110,1,1,...,0.0,3.0,39.0,11261.0,41081.0,0,NaN,72.6,412,1
3,TRAIN_00003,COMPONENT3,7389,2010,2,200,0,8,3,0,...,NaN,NaN,NaN,NaN,NaN,0,NaN,133.3,7,0
4,TRAIN_00004,COMPONENT3,3954,2015,4,200,0,1,157,0,...,NaN,NaN,NaN,NaN,NaN,0,NaN,133.1,128,0


In [5]:
test.head(5)

,ID,COMPONENT_ARBITRARY,ANONYMOUS_1,YEAR,ANONYMOUS_2,AG,CO,CR,CU,FE,H2O,MN,MO,NI,PQINDEX,TI,V,V40,ZN
0,TEST_0000,COMPONENT1,2192,2016,200,0,0,0,1,12,0.0,0,0,0,10,0,0,91.3,1091
1,TEST_0001,COMPONENT3,2794,2011,200,0,0,2,1,278,0.0,3,0,0,2732,1,0,126.9,12
2,TEST_0002,COMPONENT2,1982,2010,200,0,0,0,16,5,0.0,0,0,0,11,0,0,44.3,714
3,TEST_0003,COMPONENT3,1404,2009,200,0,0,3,4,163,0.0,4,3,0,8007,0,0,142.8,94
4,TEST_0004,COMPONENT2,8225,2013,200,0,0,0,6,13,0.0,0,0,0,16,0,0,63.4,469


In [6]:
categorical_features = ['COMPONENT_ARBITRARY', 'YEAR']
# Inference(실제 진단 환경)에 사용하는 컬럼
test_stage_features = ['COMPONENT_ARBITRARY', 'ANONYMOUS_1', 'YEAR' , 'ANONYMOUS_2', 'AG', 'CO', 'CR', 'CU', 'FE', 'H2O', 'MN', 'MO', 'NI', 'PQINDEX', 'TI', 'V', 'V40', 'ZN']

# Category encoding

In [7]:
from sklearn.preprocessing import OneHotEncoder

In [8]:
def last_num(x):
    return int(x[-1])

In [10]:
train["COMPONENT_ARBITRARY"] = train["COMPONENT_ARBITRARY"].apply(last_num)
test["COMPONENT_ARBITRARY"] = test["COMPONENT_ARBITRARY"].apply(last_num)

In [11]:
# encoder = OneHotEncoder()
# train["COMPONENT_ARBITRARY"] = encoder.fit_transform(np.array(train["COMPONENT_ARBITRARY"]).reshape(1, -1)).toarray()[0]
# test["COMPONENT_ARBITRARY"] = encoder.transform(np.array(test["COMPONENT_ARBITRARY"]).reshape(1, -1))

In [12]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14095 entries, 0 to 14094
Data columns (total 54 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   ID                   14095 non-null  object 
 1   COMPONENT_ARBITRARY  14095 non-null  int64  
 2   ANONYMOUS_1          14095 non-null  int64  
 3   YEAR                 14095 non-null  int64  
 4   SAMPLE_TRANSFER_DAY  14095 non-null  int64  
 5   ANONYMOUS_2          14095 non-null  int64  
 6   AG                   14095 non-null  int64  
 7   AL                   14095 non-null  int64  
 8   B                    14095 non-null  int64  
 9   BA                   14095 non-null  int64  
 10  BE                   14095 non-null  int64  
 11  CA                   14095 non-null  int64  
 12  CD                   12701 non-null  float64
 13  CO                   14095 non-null  int64  
 14  CR                   14095 non-null  int64  
 15  CU                   14095 non-null 

# Missing value(need to add scaling for linear model)

In [13]:
train = train.fillna(train.mean())
test = test.fillna(test.mean())

C:\Users\hjhhi\AppData\Local\Temp\ipykernel_20096\1144841304.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  train = train.fillna(train.mean())
C:\Users\hjhhi\AppData\Local\Temp\ipykernel_20096\1144841304.py:2: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  test = test.fillna(test.mean())


In [14]:
from sklearn.model_selection import train_test_split

# Split 

In [15]:
X = train.drop(['ID', 'Y_LABEL'], axis = 1)
y = train['Y_LABEL']

test = test.drop(['ID'], axis = 1)

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, stratify=y)

# Teacher model 

In [16]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier

In [23]:
teacher = XGBClassifier()
teacher.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, feature_types=None, gamma=0, gpu_id=-1,
              grow_policy='depthwise', importance_type=None,
              interaction_constraints='', learning_rate=0.300000012,
              max_bin=256, max_cat_threshold=64, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=100,
              n_jobs=0, num_parallel_tree=1, predictor='auto', random_state=0, ...)

# Eval teacher (low then base code[0.79])

In [24]:
from sklearn.metrics import f1_score

In [25]:
pred = teacher.predict(X_val)
f1_score(y_val, pred)

0.636604774535809

# Student

In [30]:
from xgboost import XGBRegressor

In [31]:
teacher_pred = teacher.predict_proba(X_train)

In [32]:
student = XGBRegressor()
student.fit(X_train[test_stage_features],teacher_pred[:,1])

XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, feature_types=None, gamma=0, gpu_id=-1,
             grow_policy='depthwise', importance_type=None,
             interaction_constraints='', learning_rate=0.300000012, max_bin=256,
             max_cat_threshold=64, max_cat_to_onehot=4, max_delta_step=0,
             max_depth=6, max_leaves=0, min_child_weight=1, missing=nan,
             monotone_constraints='()', n_estimators=100, n_jobs=0,
             num_parallel_tree=1, predictor='auto', random_state=0, ...)

In [95]:
student_pred = student.predict(X_val[test_stage_features])
student_pred = pd.Series([i for i in student_pred>0.5])
student_pred[student_pred==False]=0
student_pred[student_pred==True]=1
student_pred = student_pred.astype(int)
f1_score(student_pred, y_val)

0.08029197080291971

In [96]:
submission = student.predict(test)
submission = pd.Series([i for i in submission>0.5])
submission[submission==False]=0
submission[submission==True]=1
submission = submission.astype(int)

In [97]:
sample_submission["Y_LABEL"] = submission

In [98]:
sample_submission.to_csv('./submit.csv', index=False)